In [13]:
import pandas as pd
import os

# Path to the extracted dataset folder
folder_path = 'airbnb-prices-in-european-cities'  # Adjust as needed

# List all CSV files
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

combined_df = []

for file in csv_files:
    file_path = os.path.join(folder_path, file)

    # Load CSV
    df = pd.read_csv(file_path)

    # Extract city and weekend flag from filename
    filename_no_ext = os.path.splitext(file)[0]
    
    # Split city and weekday/weekend info
    if filename_no_ext.endswith('_weekends'):
        city = filename_no_ext.replace('_weekends', '').capitalize()
        is_weekend = True
    elif filename_no_ext.endswith('_weekdays'):
        city = filename_no_ext.replace('_weekdays', '').capitalize()
        is_weekend = False
    else:
        city = filename_no_ext.capitalize()
        is_weekend = pd.NA

    df['City'] = city
    df['Is_weekend'] = is_weekend

    combined_df.append(df)

# Concatenate all city data
final_df = pd.concat(combined_df, ignore_index=True)

# Save to a new CSV
final_df.to_csv('combined_airbnb_data.csv', index=False)


In [15]:
df = pd.read_csv('combined_airbnb_data.csv')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51707 entries, 0 to 51706
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  51707 non-null  int64  
 1   realSum                     51707 non-null  float64
 2   room_type                   51707 non-null  object 
 3   room_shared                 51707 non-null  bool   
 4   room_private                51707 non-null  bool   
 5   person_capacity             51707 non-null  float64
 6   host_is_superhost           51707 non-null  bool   
 7   multi                       51707 non-null  int64  
 8   biz                         51707 non-null  int64  
 9   cleanliness_rating          51707 non-null  float64
 10  guest_satisfaction_overall  51707 non-null  float64
 11  bedrooms                    51707 non-null  int64  
 12  dist                        51707 non-null  float64
 13  metro_dist                  517

In [19]:
import os
os.getcwd()

'C:\\Users\\dieky\\Software dev\\AI & Machine learning\\ML_exam'